<a href="https://colab.research.google.com/github/Siddharth-Singh-2004/CU-BEMS_Forecasting/blob/main/CU_BEMS_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df=pd.read_csv('/content/drive/MyDrive/2018Floor1.csv')

In [5]:
z1_Light = df['z1_Light(kW)'].tolist()

In [7]:
len(z1_Light)

264960

In [8]:
z1_Light[:10]

[12.94, 12.97, 12.97, 12.98, 13.01, 12.98, 12.97, 12.98, 12.99, 13.0]

In [9]:
df['Date']=pd.to_datetime(df['Date'])

In [11]:
new_df = df[['Date', 'z1_Light(kW)']]

In [12]:
new_df.head()

,Date,z1_Light(kW)
0,2018-07-01 00:00:00,12.94
1,2018-07-01 00:01:00,12.97
2,2018-07-01 00:02:00,12.97
3,2018-07-01 00:03:00,12.98
4,2018-07-01 00:04:00,13.01


In [13]:
df.rename(columns={'z1_Light(kW)': 'z1_Light'}, inplace=True)

In [15]:
max = df['z1_Light'].max()
min = df['z1_Light'].min()
print(f'Range: {min} to {max}')

Range: 0.0 to 134.97


In [16]:
# Normalization
normalized_z1_Light = (z1_Light - min)/(max - min)

In [17]:
normalized_z1_Light[:10]

array([0.09587316, 0.09609543, 0.09609543, 0.09616952, 0.09639179,
       0.09616952, 0.09609543, 0.09616952, 0.09624361, 0.0963177 ])

In [19]:
# Custom Dataset
class TimeSeriesDataset(Dataset):

  def __init__(self, data, sequence_length):
    self.data = data
    self.sequence_length = sequence_length

  def __len__(self):
    return len(self.data) - self.sequence_length

  def __getitem__(self, idx):
    x = self.data[idx:idx+self.sequence_length]
    y = self.data[idx+self.sequence_length]
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

